In [11]:
#из задания
C = [318668, 317632, 226697, 388930, 357448, 297811,
344670, 219717, 388930, 307414, 220516, 281175]
print(f'Шифрованное сообщение: {C}')
B = [106507, 31482, 107518, 60659, 80717, 81516, 117973, 87697]
print(f'Публичный ключ: {B}')

Шифрованное сообщение: [318668, 317632, 226697, 388930, 357448, 297811, 344670, 219717, 388930, 307414, 220516, 281175]
Публичный ключ: [106507, 31482, 107518, 60659, 80717, 81516, 117973, 87697]


In [12]:
n = len(B) #длина открытого ключа
print(H)
print(C) # перехваченное шифрованное сообщение
c = C[0]
print(c) # первая буква

#Криптоатака1 
#формируем матрицу вида
#1 0 0 0 0 0 0 0 h1
#0 1 0 0 0 0 0 0 h2
#0 0 1 0 0 0 0 0 h3
#0 0 0 1 0 0 0 0 h4
#0 0 0 0 1 0 0 0 h5
#0 0 0 0 0 1 0 0 h6
#0 0 0 0 0 0 1 0 h7
#0 0 0 0 0 0 0 1 h8
#0 0 0 0 0 0 0 0 c
#

[106507, 31482, 107518, 60659, 80717, 81516, 117973, 87697]
[318668, 317632, 226697, 388930, 357448, 297811, 344670, 219717, 388930, 307414, 220516, 281175]
318668


In [13]:
# Создаём единичную матрицу размером n x n
I = identity_matrix(ZZ, n)
print("Единичная матрица I:")
print(I)


Единичная матрица I:
[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]


In [14]:
# Создаём столбец L_h из элементов B и -C
right = matrix(ZZ, n + 1, 1, H + [-c])  # Матрица n+1 x 1
print("Правый столбец H:")
print(right)

Правый столбец H:
[ 106507]
[  31482]
[ 107518]
[  60659]
[  80717]
[  81516]
[ 117973]
[  87697]
[-318668]


In [15]:
# Добавляем строку нулей к I снизу, чтобы сделать её n+1 x n
I = I.stack(matrix(ZZ, 1, n, [0 for _ in range(n)]))
print("I с добавленной строкой:")
print(I)

I с добавленной строкой:
[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0]


In [16]:
# Объединяем I и L_h
L = I.augment(right)
#print("Матрица решётки L:")
#print(L)

In [17]:
# Применяем LLL
L_LLL = L.LLL()
print("\nМатрица после LLL:")
print(L_LLL)


Матрица после LLL:
[ 0  1  0  0  0  1  1  1  0]
[-1  1  0  1 -1 -2 -2  2  1]
[ 3  1  2 -1  1  1 -1  1  1]
[ 1 -1 -2 -1 -3 -1  1  1  1]
[ 2 -2 -1  1  0  2 -3  1  1]
[ 0  0  3 -4 -2  1  0  0  0]
[-1  3 -1  3  0  0 -1 -3  2]
[ 0 -1  1  4  0  0  0  0  4]
[-2 -1 -2 -3  1 -1  2  1  3]


In [18]:
#1
print(L_LLL[0][:-1].dot_product(vector(H)),c) #берём первую строку уменьшенного базиса, тут 
#это вектор длиной n+1 (где n — длина B), например, [x1,x2,…,xn,1], где xi — биты сообщения, а последняя координата — 1 или −1.
#убрали послед координату
#при помощи dot_product - вычисляем скалярное произведение вектора x1,x2...xn на B(открытый ключ)
bs1 = ''.join([str(x) for x in L.LLL()[0][:-1]])
print(bs1)#совпадает с первой буквой в бинарном виде
print(chr(int(bs1,2)))#расшифровали первую букву 



318668 318668
01000111
G


In [19]:
def LLL_VC_sage(C, B):
    n = len(B)
    #print("Публичный ключ B:", B)
    #print("Шифротекст C:", C)
    binary_result = ""  # Собираем все биты
    for c in C:
        I = identity_matrix(ZZ, n) #создаём матрицу
        right = matrix(ZZ, n + 1, 1, B + [-c]) #формируем правый столбец
        I = I.stack(matrix(ZZ, 1, n, [0 for _ in range(n)])) #добавляем строку нулей
        L = I.augment(right) #присоединяем правый столбец
        L_LLL = L.LLL() #LLL
        bs1 = "".join("1" if x > 0 else "0" for x in L_LLL[0][:-1])
        print(f"Бинарный код для c={c}: {bs1}")
        binary_result += bs1
    # Преобразуем бинарную строку в текст по 8 бит
    message = ""
    for i in range(0, len(binary_result) - 7, 8):  # Берем только полные 8-битные куски
        byte = binary_result[i:i+8]
        if int(byte, 2) != 0:  # Игнорируем padding
            message += chr(int(byte, 2))
    print("Расшифрованное сообщение:", message)

In [20]:
LLL_VC_sage(C,H)

Бинарный код для c=318668: 01000111
Бинарный код для c=317632: 01110010
Бинарный код для c=226697: 01100001
Бинарный код для c=388930: 01101101
Бинарный код для c=357448: 00101101
Бинарный код для c=297811: 01010011
Бинарный код для c=344670: 01100011
Бинарный код для c=219717: 01101000
Бинарный код для c=388930: 01101101
Бинарный код для c=307414: 01101001
Бинарный код для c=220516: 01100100
Бинарный код для c=281175: 01110100
Расшифрованное сообщение: Gram-Schmidt
